# CH14 Exercises

### 10. Use transfer learning for large image classification, going through these steps:


 a. Create a training set containing at least 100 images per class. For example, you



 could classify your own pictures based on the location (beach, mountain, city,



 etc.), or alternatively you can use an existing dataset (e.g., from TensorFlow



 Datasets).

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split





tf.random.set_seed(42)

In [ ]:
(train_ds, valid_ds, test_ds), ds_info = tfds.load(
                        'beans',
                        split = ['train','validation','test'],
                        shuffle_files=True,
                        with_info=True, as_supervised=True
                        )

In [ ]:
print(f"Training set size: {ds_info.splits['train'].num_examples }")
print(f"Validation set size: {ds_info.splits['validation'].num_examples }")
print(f"Test set size: {ds_info.splits['test'].num_examples}")

In [ ]:
train_ds.cardinality().numpy()

In [ ]:
type(test_ds)

In [ ]:
ds_info

In [ ]:
for element, label in train_ds.take(5):
    print(element.shape, label)

In [ ]:
fig = tfds.show_examples(test_ds.take(10),ds_info)

c. Build the input pipeline, apply the appropriate preprocessing operations, and



optionally add data augmentation.

In [ ]:
samples = next(iter(train_ds))
images = samples[0]
labels = samples[1]



print("Label: ",labels)
print("Images Shape: ",images.shape)

---

In [ ]:
from tensorflow.keras.applications.convnext import preprocess_input, ConvNeXtBase, ConvNeXtSmall

In [ ]:
IMG_SIZE = 224 


def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    return image, label

In [ ]:
prepared = preprocess(images,labels)
prepared[0].shape, prepared[0].dtype, np.max(prepared[0])

In [ ]:
train_dataset = train_ds.map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)


valid_dataset = valid_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


test_dataset = test_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = ConvNeXtBase(weights='imagenet',include_top=True)

In [ ]:
Y_proba = model.predict(test_dataset.take(3))
Y_proba.shape

In [ ]:
top_K = tf.keras.applications.convnext.decode_predictions(Y_proba, top=3)

for image_index in range(3):
     print(f"Image #{image_index}")
     for class_id, name, y_proba in top_K[image_index]:
         print(f"  {class_id} - {name:12s} {y_proba:.2%}")

---

d. Fine-tune a pretrained model on this dataset.

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
base_model = ConvNeXtSmall(weights='imagenet',include_top=False,input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model

In [ ]:
augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal",input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.RandomRotation(0.5),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(.1)])

In [ ]:
n_classes = ds_info.features['label'].num_classes
n_classes

In [ ]:
base_model.summary()

### Transfer Learning without augmenatation layers

In [ ]:
model = tf.keras.models.Sequential([
#     augmentation_layers,
    base_model,  

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('leaky_relu'),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(n_classes, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)# only 3 because it is in training the top layer phase now.
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('beans_model.keras',save_best_only=True,verbose=1)



adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,epochs=150 ,validation_data=valid_dataset, callbacks=[early_stopping,model_checkpoint])

---

In [ ]:
loss, acc= model.evaluate(test_dataset)
print(f'Loss: {loss}\nAccuracy: {acc}')

---

In [ ]:
tf.keras.backend.clear_session()


### Transfer Learning with augmenatation layers

In [ ]:
augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomRotation(0.3),
#     tf.keras.layers.RandomContrast(.1)]
)

for layer in base_model.layers:
    layer.trainable = False
    

model_aug = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    augmentation_layers,
    base_model,  

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('leaky_relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(128, kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('leaky_relu'),
    tf.keras.layers.Dropout(0.1),

    tf.keras.layers.Dense(n_classes, activation='softmax')
])

def scheduler(epoch, lr):
    if epoch > 10:  # Start reducing the learning rate after 10 epochs
        return float(lr * np.exp(-0.1))  # Ensure it's returned as a float
    return float(lr)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('beans_model_aug.keras',save_best_only=True,verbose=1)



adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_aug.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])


history = model_aug.fit(train_dataset,epochs=150 ,
                        validation_data=valid_dataset,
                        callbacks=[early_stopping,model_checkpoint,lr_scheduler])

In [ ]:
loss, acc= model_aug.evaluate(test_dataset)
print(f'Loss: {loss}\nAccuracy: {acc}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = np.concatenate([y for x, y in test_dataset], axis=0)  
y_pred = np.argmax(model_aug.predict(test_dataset), axis=-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='.4f', cmap='Blues', xticklabels=range(n_classes), yticklabels=range(n_classes))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model_aug.layers

In [ ]:
for layer in model_aug.layers:
    print(layer.name , layer.trainable)

In [ ]:


early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('_beans_model.keras',save_best_only=True,verbose=1)



model_aug.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
    loss='sparse_categorical_crossentropy',
    metrics=[
        'accuracy'
    ]
)

In [ ]:
history = model_aug.fit(train_dataset,epochs=150 ,validation_data=valid_dataset,
                    callbacks=[early_stopping,model_checkpoint, lr_scheduler]
                   )

In [ ]:
train_loss = history.history['loss']

val_loss = history.history['val_loss']

train_accuracy = history.history['accuracy']

val_accuracy = history.history['val_accuracy']



fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))



ax1.plot(train_loss, label='Training Loss')

ax1.plot(val_loss, label='Validation Loss')

ax1.set_title('Training and Validation Loss')

ax1.set_xlabel('Epoch')

ax1.set_ylabel('Loss')

ax1.legend()



ax2.plot(train_accuracy, label='Training Accuracy')

ax2.plot(val_accuracy, label='Validation Accuracy')

ax2.set_title('Training and Validation Accuracy')

ax2.set_xlabel('Epoch')

ax2.set_ylabel('Accuracy')

ax2.legend()



plt.show()

In [ ]:
loss, acc= model_aug.evaluate(test_dataset)
print(f'Loss: {loss}\nAccuracy: {acc}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = np.concatenate([y for x, y in test_dataset], axis=0)  
y_pred = np.argmax(model_aug.predict(test_dataset), axis=-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='.4f', cmap='Blues', xticklabels=range(n_classes), yticklabels=range(n_classes))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:


early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('ex_beans_model.keras',save_best_only=True,verbose=1)



model_aug.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
    loss='sparse_categorical_crossentropy',
    metrics=[
        'accuracy'
    ]
)

In [ ]:
history_ex = model_aug.fit(train_dataset,epochs=150 ,validation_data=test_dataset,
                    callbacks=[early_stopping,model_checkpoint]
                   )

In [ ]:
loss, acc= model_aug.evaluate(test_dataset)
print(f'Loss: {loss}\nAccuracy: {acc}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = np.concatenate([y for x, y in test_dataset], axis=0)  
y_pred = np.argmax(model_aug.predict(test_dataset), axis=-1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='.4f', cmap='Blues', xticklabels=range(n_classes), yticklabels=range(n_classes))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model_aug.summary()

In [ ]:
loaded_model = tf.keras.models.load_model('ex_beans_model.keras')


In [ ]:
loaded_model.predict(test_dataset)